In [1]:
import pandas as pd
from datetime import datetime
import gzip


## Con el objetivo de aumentar el rendimiendo de la API , al tratarse con un dataset muy grande, se realizaran nuevos dataframe orientados ar esponder consultas de funciones especificas, estos se guardaran en csv y luego seran usados por las funciones


In [2]:
with gzip.open('datasets/user_items_proc.csv.gz', 'rb') as f:
    user_items_proc = pd.read_csv(f, encoding='utf-8')
    
steam_games = pd.read_csv('./datasets/steam_games.csv', parse_dates=['release_date'])
user_review = pd.read_csv('./datasets/user_reviews.csv')

### Funcion  userdata : Genero un data frame con 4 columnas:
* user_id : con valores unicos.
* intems_count: Cantidad de items.
* price : Total gastado.
* porcentaje_recomendacion_true: Porcentaje de recommend = True frente al total de rocommend

### Luego se guarda en csv y se usara para la funcion

In [25]:


diner_user = pd.merge(steam_games[['app_name','price']],user_items_proc[['user_id','items_count','item_name']],left_on='app_name',right_on='item_name',how='inner')
users_items = diner_user.groupby('user_id').agg(
    {
       'items_count':'first',
        'price':'sum'
    }
).reset_index()

recomendaciones_true = user_review.groupby('user_id')['recommend'].sum()
total_registros_por_usuario = user_review.groupby('user_id')['recommend'].count()
porcentaje_recomendaciones_true = (recomendaciones_true / total_registros_por_usuario) * 100
users_true_reviews = pd.DataFrame({
    'user_id': porcentaje_recomendaciones_true.index,
    'porcentaje_recomendaciones_true': porcentaje_recomendaciones_true.values
})
items_reviews_users = pd.merge(users_items,users_true_reviews,on='user_id',how='left')
items_reviews_users.to_csv('./datasets/items_reviews_users.csv',index=False)

In [27]:
items_reviews_users

,user_id,items_count,price,porcentaje_recomendaciones_true
0,--000--,58,332.83,100.0
1,--ace--,44,122.86,100.0
2,--ionex--,23,69.95,100.0
3,-2SV-vuLB-Kg,68,340.55,100.0
4,-Azsael-,167,1349.23,100.0
...,...,...,...,...
22564,zwanzigdrei,93,519.60,100.0
22565,zy0705,6,0.00,100.0
22566,zynxgameth,50,285.11,100.0
22567,zyr0n1c,73,481.65,100.0


### Funcion  UserForGenre : Genero un data frame con 3 columnas:
* Género : Cada uno de los generos.
* Usuario : El usuario que acumula mas horas para ese genero.
* Año_Horas = Una lista de diccionarios con llave como año y cantidad de horas para ese año como valor


### Luego se guarda en csv y se usara para la funcion

In [26]:
generos = steam_games.drop(columns=['app_name','release_date','specs','price','id','developer','Year','Accounting'])
gener_user = pd.merge(user_items_proc[['item_name','playtime_forever','user_id']],steam_games,left_on='item_name',right_on='app_name',how='inner')
resultados_por_genero = []
for genero in generos.columns:
    genero_data = gener_user[gener_user[genero] == 1]
    
    if not genero_data.empty:
        user_max = genero_data.groupby('user_id')['playtime_forever'].sum().idxmax()
        result_df = genero_data[genero_data['user_id'] == user_max].groupby('Year')['playtime_forever'].sum().reset_index()
        formatted_result = [{'Año': record['Year'], 'Horas': record['playtime_forever']} for _, record in result_df.iterrows()]
        resultados_por_genero.append({'Género': genero, 'Usuario': user_max, 'Año_Horas': formatted_result})
    else:
        resultados_por_genero.append({'Género': genero, 'Usuario': None, 'Año_Horas': []})

df_resultados = pd.DataFrame(resultados_por_genero)

df_resultados.dropna(inplace=True)
df_resultados.to_csv('./datasets/max_por_gen.csv',index=False)

In [28]:
df_resultados

,Género,Usuario,Año_Horas
0,Action,shinomegami,"[{'Año': 1988.0, 'Horas': 136.0}, {'Año': 1991..."
1,Adventure,REBAS_AS_F-T,"[{'Año': 1998.0, 'Horas': 1.0}, {'Año': 1999.0..."
2,Animation &amp; Modeling,76561198059330972,"[{'Año': 2013.0, 'Horas': 65427.0}]"
3,Audio Production,KJ72,"[{'Año': 2014.0, 'Horas': 18940.0}]"
4,Casual,REBAS_AS_F-T,"[{'Año': 1999.0, 'Horas': 0.0}, {'Año': 2002.0..."
5,Design &amp; Illustration,Xyphien,"[{'Año': 2012.0, 'Horas': 64657.0}, {'Año': 20..."
6,Early Access,76561197978756659,"[{'Año': 2013.0, 'Horas': 316786.0}, {'Año': 2..."
7,Education,tsunamitad,"[{'Año': 1994.0, 'Horas': 0.0}, {'Año': 2010.0..."
8,Free to Play,idonothack,"[{'Año': 1999.0, 'Horas': 0.0}, {'Año': 2003.0..."
9,Indie,REBAS_AS_F-T,"[{'Año': 1999.0, 'Horas': 0.0}, {'Año': 2001.0..."


In [ ]:
user_review['posted'].iloc[0].split(',')[-1].strip(' .')